In [ ]:
%%html
<div id="ar_final_box" style="text-align: center; background: #1a1a1a; padding: 20px; border-radius: 15px; color: white; font-family: sans-serif;">
    <h2 style="color: #3498db;">PROYECTO FINAL: Filtros AR 3D</h2>
    
    <div style="margin-bottom: 10px; display: flex; justify-content: center; gap: 5px; flex-wrap: wrap;">
        <button onclick="setF(0)">❌ Quitar</button>
        <button onclick="setF(1)">🔴 Nariz Roja</button>
        <button onclick="setF(2)">🤠 Sombrero</button>
        <button onclick="setF(3)">😎 Lentes</button>
        <button onclick="setF(4)">👑 Corona</button>
    </div>

    <div style="margin-bottom: 15px;">
        <button id="m_btn" onclick="togM()" style="padding: 8px 15px; background: #8e44ad; color: white; border: none; border-radius: 5px; cursor: pointer;">Ocultar Malla Verde</button>
    </div>

    <div style="position: relative; display: inline-block;">
        <video id="v_src" style="display:none" playsinline></video>
        <canvas id="c_out" width="640" height="480" style="background: #000; border: 2px solid #444; border-radius: 10px;"></canvas>
    </div>
    
    <div style="margin-top: 15px;">
        <button id="b_start" onclick="runAr()" style="padding: 15px 30px; background: #27ae60; color: white; border: none; border-radius: 10px; cursor: pointer; font-weight: bold;">🚀 ACTIVAR FILTROS AR</button>
        <button onclick="detenerTodo()" style="padding: 15px 30px; background: #e74c3c; color: white; border: none; border-radius: 10px; cursor: pointer; margin-left: 10px; display: none;" id="b_stop">🛑 APAGAR</button>
        <p id="debug_log" style="color: #f1c40f; font-size: 13px; margin-top: 10px; background: #000; padding: 5px;">Estado: Esperando clic...</p>
    </div>
</div>

<script src="https://cdn.jsdelivr.net/npm/@mediapipe/camera_utils/camera_utils.js" crossorigin="anonymous"></script>
<script src="https://cdn.jsdelivr.net/npm/@mediapipe/face_mesh/face_mesh.js" crossorigin="anonymous"></script>
<script src="https://cdn.jsdelivr.net/npm/@mediapipe/drawing_utils/drawing_utils.js" crossorigin="anonymous"></script>

<script>
var vid_el = document.getElementById('v_src');
var can_el = document.getElementById('c_out');
var ctx_el = can_el.getContext('2d');
var log_el = document.getElementById('debug_log');

var show_m = true;
var filter_id = 0;
var arCamera = null;

function setF(n) {
    filter_id = n;
    log_el.innerText = n > 0 ? "Filtro " + n + " activado ✓" : "Filtro removido";
}

function togM() {
    show_m = !show_m;
    document.getElementById('m_btn').innerText = show_m ? "Ocultar Malla Verde" : "Mostrar Malla Verde";
}

// FUNCIONES PARA DIBUJAR FILTROS
function drawClownNose(ctx, x, y, size) {
    ctx.save();
    ctx.shadowColor = 'rgba(0,0,0,0.4)';
    ctx.shadowBlur = 25;
    ctx.shadowOffsetX = 15;
    ctx.shadowOffsetY = 15;
    
    const gradient = ctx.createRadialGradient(x - size*0.2, y - size*0.2, 0, x, y, size*0.6);
    gradient.addColorStop(0, '#ff6b6b');
    gradient.addColorStop(0.5, '#ee4444');
    gradient.addColorStop(1, '#cc2222');
    
    ctx.fillStyle = gradient;
    ctx.beginPath();
    ctx.arc(x, y, size*0.25, 0, Math.PI * 2);
    ctx.fill();
    
    ctx.shadowColor = 'transparent';
    const highlightGrad = ctx.createRadialGradient(x - size*0.15, y - size*0.15, 0, x - size*0.15, y - size*0.15, size*0.25);
    highlightGrad.addColorStop(0, 'rgba(255,255,255,0.9)');
    highlightGrad.addColorStop(1, 'rgba(255,255,255,0)');
    ctx.fillStyle = highlightGrad;
    ctx.beginPath();
    ctx.arc(x - size*0.15, y - size*0.15, size*0.25, 0, Math.PI * 2);
    ctx.fill();
    
    ctx.fillStyle = 'rgba(255,255,255,0.6)';
    ctx.beginPath();
    ctx.arc(x + size*0.1, y + size*0.15, size*0.08, 0, Math.PI * 2);
    ctx.fill();
    ctx.restore();
}

function drawHat(ctx, x, y, size) {
    ctx.save();
    ctx.fillStyle = '#8B4513';
    ctx.strokeStyle = '#654321';
    ctx.lineWidth = 3;
    ctx.shadowColor = 'rgba(0,0,0,0.5)';
    ctx.shadowBlur = 15;
    
    ctx.beginPath();
    ctx.ellipse(x, y, size*0.8, size*0.25, 0, 0, Math.PI * 2);
    ctx.fill();
    ctx.stroke();
    
    ctx.fillStyle = '#A0522D';
    ctx.beginPath();
    ctx.moveTo(x - size*0.4, y);
    ctx.lineTo(x - size*0.35, y - size*0.8);
    ctx.lineTo(x + size*0.35, y - size*0.8);
    ctx.lineTo(x + size*0.4, y);
    ctx.closePath();
    ctx.fill();
    ctx.stroke();
    
    ctx.fillStyle = '#FFD700';
    ctx.fillRect(x - size*0.35, y - size*0.3, size*0.7, size*0.15);
    ctx.restore();
}

function drawGlasses(ctx, x, y, size) {
    ctx.save();
    ctx.strokeStyle = '#000000';
    ctx.lineWidth = 4;
    ctx.shadowColor = 'rgba(0,0,0,0.5)';
    ctx.shadowBlur = 10;
    
    ctx.fillStyle = 'rgba(0,0,0,0.3)';
    ctx.beginPath();
    ctx.arc(x - size*0.35, y, size*0.25, 0, Math.PI * 2);
    ctx.fill();
    ctx.stroke();
    
    ctx.beginPath();
    ctx.arc(x + size*0.35, y, size*0.25, 0, Math.PI * 2);
    ctx.fill();
    ctx.stroke();
    
    ctx.beginPath();
    ctx.moveTo(x - size*0.1, y);
    ctx.lineTo(x + size*0.1, y);
    ctx.stroke();
    
    ctx.fillStyle = 'rgba(255,255,255,0.6)';
    ctx.beginPath();
    ctx.arc(x - size*0.42, y - size*0.08, size*0.08, 0, Math.PI * 2);
    ctx.fill();
    ctx.beginPath();
    ctx.arc(x + size*0.28, y - size*0.08, size*0.08, 0, Math.PI * 2);
    ctx.fill();
    ctx.restore();
}

function drawCrown(ctx, x, y, size) {
    ctx.save();
    ctx.fillStyle = '#FFD700';
    ctx.strokeStyle = '#FFA500';
    ctx.lineWidth = 3;
    ctx.shadowColor = 'rgba(0,0,0,0.5)';
    ctx.shadowBlur = 15;
    
    ctx.beginPath();
    ctx.moveTo(x - size*0.5, y);
    ctx.lineTo(x - size*0.4, y - size*0.4);
    ctx.lineTo(x - size*0.25, y - size*0.2);
    ctx.lineTo(x, y - size*0.5);
    ctx.lineTo(x + size*0.25, y - size*0.2);
    ctx.lineTo(x + size*0.4, y - size*0.4);
    ctx.lineTo(x + size*0.5, y);
    ctx.closePath();
    ctx.fill();
    ctx.stroke();
    
    const jewels = [
        {x: x - size*0.4, y: y - size*0.4, c: '#ff0000'},
        {x: x - size*0.25, y: y - size*0.2, c: '#00ff00'},
        {x: x, y: y - size*0.5, c: '#ff0000'},
        {x: x + size*0.25, y: y - size*0.2, c: '#0000ff'},
        {x: x + size*0.4, y: y - size*0.4, c: '#ff00ff'}
    ];
    
    jewels.forEach(j => {
        ctx.fillStyle = j.c;
        ctx.beginPath();
        ctx.arc(j.x, j.y, size*0.05, 0, Math.PI * 2);
        ctx.fill();
    });
    ctx.restore();
}

async function runAr() {
    log_el.innerText = "Paso 1: Solicitando cámara...";
    
    // Detener otros filtros
    detenerTodo();
    
    try {
        const stream = await navigator.mediaDevices.getUserMedia({ video: { width: 640, height: 480 } });
        window.cameraManager.currentStream = stream;
        window.cameraManager.currentFilter = 'ar';
        
        vid_el.srcObject = stream;
        await vid_el.play();
        
        document.getElementById('b_start').style.display = 'none';
        document.getElementById('b_stop').style.display = 'inline-block';
        
        log_el.innerText = "Paso 2: Cámara activa. Cargando IA...";
        setTimeout(startIA, 500);
    } catch (e) {
        log_el.innerText = "ERROR: No se pudo abrir la cámara. ¿Diste permiso?";
        console.error(e);
    }
}

function startIA() {
    try {
        const mesh = new FaceMesh({
            locateFile: (f) => `https://cdn.jsdelivr.net/npm/@mediapipe/face_mesh/${f}`
        });
        
        mesh.setOptions({ 
            maxNumFaces: 1, 
            refineLandmarks: true, 
            minDetectionConfidence: 0.5,
            minTrackingConfidence: 0.5
        });
        
        mesh.onResults((res) => {
            if (window.cameraManager.currentFilter !== 'ar') return;
            
            ctx_el.save();
            ctx_el.clearRect(0, 0, can_el.width, can_el.height);
            ctx_el.drawImage(res.image, 0, 0, can_el.width, can_el.height);
            
            if (res.multiFaceLandmarks && res.multiFaceLandmarks.length > 0) {
                const face = res.multiFaceLandmarks[0];
                
                if (show_m) {
                    drawConnectors(ctx_el, face, FACEMESH_TESSELATION, {color: '#00FF0050', lineWidth: 1});
                }

                const faceW = Math.abs(face[454].x - face[234].x) * can_el.width;
                
                if (filter_id === 1) {
                    const nose = face[1];
                    drawClownNose(ctx_el, nose.x * can_el.width, nose.y * can_el.height, faceW * 0.15);
                } 
                else if (filter_id === 2) {
                    const top = face[10];
                    drawHat(ctx_el, top.x * can_el.width, (top.y * can_el.height) - faceW*0.6, faceW * 0.8);
                }
                else if (filter_id === 3) {
                    const eyes = face[168];
                    drawGlasses(ctx_el, eyes.x * can_el.width, eyes.y * can_el.height, faceW);
                }
                else if (filter_id === 4) {
                    const top = face[10];
                    drawCrown(ctx_el, top.x * can_el.width, (top.y * can_el.height) - faceW*0.5, faceW * 0.6);
                }
            }
            ctx_el.restore();
        });

        arCamera = new Camera(vid_el, {
            onFrame: async () => {
                if (window.cameraManager.currentFilter === 'ar') {
                    await mesh.send({image: vid_el});
                }
            },
            width: 640, 
            height: 480
        });
        
        arCamera.start();
        log_el.innerText = "✅ ¡Todo listo! Selecciona un filtro";
        
    } catch (err) {
        log_el.innerText = "ERROR de IA: " + err.message;
        console.error(err);
    }
}
</script>